In [1]:
# import the necessary packages
import argparse
import datetime
import imutils
import time
import cv2
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video", default = 'video_samples/P1022142.mp4', help="path to the video file")
ap.add_argument("-a", "--min_area", type=int, default=500, help="minimum area size")
args = ap.parse_args(args=[])


if args.video:
    cap = cv2.VideoCapture(args.video)
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter('videoOutput.mp4',fourcc,30.0,(640,480))
    print("Reading video file")
else:
    cap = cv2.VideoCapture(0)
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter('webcamOutput.mp4',fourcc,30.0,(640,480))
    print("Reading from webcam")
    time.sleep(1)

firstFrame = None
# loop over the frames of the video
while cap.isOpened():
    # grab the current frame and initialize the occupied/unoccupied text
    ret, frame = cap.read()
    # if the frame could not be grabbed, then we have reached the end of the video
    if not ret:
        break
    # resize the frame to 300x300 pixels, convert it to grayscale, and blur it
#     blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    frame = cv2.resize(frame, (500,500))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    text = "NO MOTION"
    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue
    # compute the absolute difference between the current frame and first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 85, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < args.min_area:
            continue
        # compute the bounding box for the contour, draw it on the frame,and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        text = "MOTION DETECTED"
    # draw the text and timestamp on the frame
    cv2.putText(frame, "{}".format(text), (10, 20), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 255), 2)
    # show the frame and record if the user presses a key   
    cv2.imshow("Frame Delta", frameDelta)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Live Feed", frame)
#     cv2.resizeWindow("Live Feed", 900,700)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break
# cleanup the camera and close any open windows
if args.video is not None:
    cap.release()
    print("Video Stopped!")
else:
    cap.release()
# out.release()
cv2.destroyAllWindows()

Reading video file
Video Stopped!
